<a href="https://colab.research.google.com/github/Kevinfhu/sistemas-de-informaci-n/blob/main/2_5_2_5_Lab_Internet_Meter_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Título](Images/cisco.png)

# Práctica de laboratorio: contador SQL de Internet


### Objetivos

<b>
* Parte 1: Configurar su base de datos
* Parte 2: Conectarse a la base de datos 
* Parte 3: Manipular los datos con pandas
* Parte 4: Unir las tablas con pandas
</b>

### Aspectos básicos/situación
En esta práctica de laboratorio aprenderá cómo conectarse a una base de datos relacional, extraer y manipular los datos a través de una notebook de Jupyter. Los datos que se le proporcionan contienen distintas mediciones para la velocidad de ping, carga y descarga en diferentes autoridades locales alrededor de Inglaterra. De momento, los datos se almacenan en un archivo .csv, pero creará una base de datos y la completará con la medición. 

Usted podrá:
* Trabajar con una base de datos relacional
* Configurar SQLite
* Crear una nueva base de datos
* Utilizar csvsql para crear una nueva tabla y recuperar datos directamente de un archivo csv grande
* Conectarse a una base de datos con la biblioteca de Python sqlite3
* Realizar consultas de SQL básicas mediante Python
* Explorar la interacción entre pandas y una base de datos SQL
* Combinar dos tablas

### Recursos necesarios
* 1 computadora con acceso a Internet
* Raspberry Pi versión 2 o superior
* Bibliotecas de Python: sqlite3, pandas, matplotlib
* Bibliotecas adicionales: csvkit
* Archivos de datos: LA_wifi_speed_UK.csv, LA_population.csv

### Actualización de SQL
* Introducción muy breve a las bases de datos relacionales (temporales): http://searchsqlserver.techtarget.com/definition/relational-database</li>
* Más videos sobre bases de datos relacionales: https://www.youtube.com/watch?v=jyju2P-7hPA&list=PLAwxTw4SYaPm4R6j_wzVOCV9fJaiQDYx4</li>
* Introducción a SQL: http://www.w3schools.com/sql/sql_intro.asp</li>

## Parte 1: Configurar su base de datos

En esta parte, utilizará SQLite, una biblioteca de procesos que implementa un motor de bases de datos de SQL transaccionales autónomo, sin servidor, sin configuración para configurar el entorno.

#### Paso 1: Configurar el entorno de trabajo.
Necesitaremos 'sqlite3' y 'csvkit' para completar esta computadora portátil. 

###### a) Instale SQLite.
En una computadora portátil de Jupyter, se pueden ejecutar los mismos comandos que se usarían en el terminal. Para hacerlo, comience la línea con un signo de exclamación (```!```).

In [ ]:
# Code Cell 1
!apt-get update
!apt-get -y install sqlite3

Después de que el proceso de instalación finalice, cree una base de datos:

    $sqlite3 <name of your database> ".databases"

*nombre de la base de datos* es el nombre que desea proporcionar a su base de datos, por ejemplo, InternetSpeed:

    $sqlite3 InternetSpeed.db ".databases"
    
Este código crea la base de datos en una ventana del terminal. Si está utilizando el terminal, asegúrese de navegar al mismo directorio en el cual reside este laboratorio.

In [ ]:
# Code Cell 2
#Create the InternetSpeed database


En este punto, en su equipo local, cuenta con una base de datos nueva y vacía llamada InternetSpeed.
Por lo general, si desea crear una tabla, necesita proporcionar un esquema que especifique el nombre de la tabla, el nombre de las columnas y el tipo de datos que van en cada columna (int, str, decimal, etc.).
<p>A los fines de este curso, se utiliza un conjunto de datos **synthetic** con las mediciones de la velocidad de WiFi registradas en 300 ubicaciones en el Reino Unido. El archivo se estructura de manera que cada columna contenga las mediciones en una ubicación de una de las tres estadísticas (ping, descarga y carga). Esto significa que la tabla que necesitamos tiene 300\*3 columnas (además de la hora y la fecha, y una columna de índice). Escribir el esquema para dicha tabla a mano es poco práctico. Afortunadamente, hay maneras de superar este inconveniente, y una de ellas es utilizar el paquete **csvkit**, que contiene la función **csvsql** la cual se puede utilizar para crear una tabla e importar los datos en un comando.</p>

###### b) Instale csvkit.
Instale csvkit en el terminal:

    $pip install csvkit
    
También puede permanecer en el entorno de la computadora portátil, como lo hizo anteriormente, agregando el signo de exclamación al comienzo de la línea.

El archivo utilizado en esta parte de la práctica de laboratorio se denomina LA_wifi_speed_UK.csv.

In [ ]:
# Code Cell 3
#Installing csvkit
!pip install csvkit

###### c) Importe los datos en la base de datos.
Utilice csvsql para crear una nueva tabla de la base de datos InternetSpeed y recupere los datos contenidos en el archivo csv. Primero, asegúrese de que no haya otra base de datos con el mismo nombre. Si existe una, elimínela:
    
    $test tmp/InternetSpeed.db && rm InternetSpeed.db

Utilice csvkit para completar la base de datos del archivo csv:

    $csvsql --db sqlite:///InternetSpeed.db --insert ./Data/LA_wifi_speed_UK.csv
    
Puede hacerlo a través del terminal o puede permanecer en el entorno de la computadora portátil.

In [ ]:
# Code Cell 4
# Removing database if already existing
!test -e InternetSpeed.db && rm InternetSpeed.db
# Importing the LA_wifi_speed_UK.csv file into a sql database file InternetSpeed.db
!csvsql --db sqlite:///InternetSpeed.db --insert ./Data/LA_wifi_speed_UK.csv
# This will take a while 

## Parte 2: Conectarse a la base de datos 

Python se utiliza para conectarse a la base de datos y ejecutar algunas consultas para saber cómo lucen estos datos.

Importe las bibliotecas:


In [ ]:
# Code Cell 5
# this is the library to talk to the database
import sqlite3
import pandas as pd
# this is a library for data visualization
from matplotlib import pyplot as plt
# this is to say that we want our plots to be part of this page, and not opened in a new window
%matplotlib inline

Puede encontrar sqlite3 en la documentación de Python aquí: https://docs.python.org/2/library/sqlite3.html

#### Paso 1: Conecte y ejecute una consulta.
Una conexión abierta y un cursor son necesarios para comunicarse con la base de datos. El cursor es una estructura de control que nos permitirá navegar por las tablas y los registros. 

Para abrir una conexión, ejecute el método ‘connect’ de la biblioteca ‘sqlite3’. Esta función toma una cadena que contiene el nombre de la base de datos para la cual la conexión es necesaria.

En este caso, el comando será:

    sqlite3.connect('InternetSpeed.db')

Esta función regresa la conexión que se almacenará como variable, llamada ‘conn’. Para crear el cursor, ejecute el método ‘cursor()’ de la siguiente manera:

    conn.cursor()

Guárdelo en una variable llamada ‘cur’.
 

###### a) Conéctese a la base de datos que acaba de crear, y luego cree un cursor que apunte a la base de datos.


In [ ]:
# Code Cell 6
# what is the name of the database you want to connect to?
# conn = sqlite3.connect(??)
# cur = conn.?()

Para ejecutar una consulta en la base de datos, necesitaremos utilizar el cursor que acabamos de crear y el método ‘execute()’. Este método toma como entrada una cadena que contiene la consulta que nos interesa. Podemos guardar la cadena en una variable; llamémosla ‘query’. Ejecute el método de esta manera:

    cur.execute(query)

La tabla sobre la que queremos consultar no tiene una excelente visualización, así que SELECCIONAREMOS sólo la columna DateTime, que contiene la fecha y la hora de las mediciones. LIMITAREMOS la cantidad de filas extraídas por la consulta a 10.

###### b) Ejecute una consulta. 
Seleccione las primeras 10 filas de la columna DateTime. 

Sugerencia: Si desea revisar el nombre de las tablas en su base de datos, vuelva a la línea de comandos interactiva y utilice este código:
    
    .tables

In [ ]:
# Code Cell 7
# complete the query with the field you want to select, the table you want to select the field from,
# and the field you want your results to be ordered by
# query = 'SELECT ? FROM ? ORDER BY ? LIMIT ?;'
# Execute the query
# cur.?(?)


El resultado de la consulta se almacena en ‘cur’. Necesitamos iterar sobre ella para ver el resultado.

In [ ]:
# Code Cell 8
for row in cur:
    print(row) 

**Nota**: Necesitamos ordenar la tabla que contiene la medición según la columna DateTime. Esto es porque SQL no posee un orden intrínseco de las filas, y estamos lidiando con una serie de tiempo, que sin el orden correcto, no tiene sentido. La selección de una muestra aleatoria de filas quitaría el componente temporal:


In [ ]:
# Code Cell 9
query = 'SELECT DateTime FROM LA_wifi_speed_UK ORDER BY RANDOM() LIMIT 10'
cur.execute(query)

for row in cur:
    print(row) 

#### Paso 2: Enfóquese en una porción de los datos.

¿Cuáles son los nombres de las 900+ columnas? Es posible recuperarlos automáticamente y guardarlos en una lista. Será práctico para distintas cosas más adelante.

En primer lugar, necesitamos ejecutar una consulta sobre todas las columnas. En este caso, no es importante la cantidad de filas que la consulta devuelve, de modo que la limitaremos a 1. 

In [ ]:
# Code Cell 10
# we need to have a cursor pointing to a table for this operation
query = 'SELECT * FROM LA_wifi_speed_UK LIMIT 1'
cur.execute(query)

El nombre de las columnas de la tabla se almacena en el campo ‘description’ del cursor.

In [ ]:
# Code Cell 11
#visualizing the first 10 rows of the field description
cur.description[:10]

Podemos dar vuelta los miembros de este campo e ingresar los nombres de las columnas en una lista (una estructura de datos práctica, en este caso).
En Python, es posible generar listas como el resultado de un bucle. Este concepto se denomina comprensión de la lista y es sólo otro nombre para un bucle más largo. Cada elemento de la lista de resultados es la operación que haría en el bucle, mientras itera sobre cierto conjunto.  

In [ ]:
# Code Cell 12
# loop over the header and store them in a list
columns = [member[0] for member in cur.description]

# ignore index column
columns = columns[1:]

Las columnas contienen las mediciones de las tres velocidades para cada área geográfica. Se establecen de esta manera: 

`<area name>_<suffix>`, 

El sufijo es ‘p’ para ping, ‘u’ para carga y ‘d’ para descarga. 

Necesitamos librarnos de los sufijos, porque estamos interesados en los nombres del área, y de momento cada nombre se repite tres veces. Utilizaremos el nombre de las áreas para mostrar las distintas mediciones y para generar una nueva tabla más adelante en la práctica de laboratorio.

Podemos reemplazar la parte final de la cadena mediante un carácter vacío. Para hacerlo, podemos utilizar el método ‘replace()’ (consulte la documentación de la cadena, https://docs.python.org/2/library/string.html).
Este método toma como entrada la parte de la cadena que se reemplazará y la nueva parte de la cadena que debe reemplazar. Por ejemplo, para quitar los caracteres ‘e’ y ‘s’ de la cadena ‘chestnut’, utilice este código:
    
    'chestnut'.replace('es', '')

###### a) Obtenga los nombres de área.
Utilice la comprensión de lista para dar vuelta la lista que contiene los nombres de las columnas de la tabla y quite los sufijos "_p", "_d" y "_u".

In [ ]:
# Code Cell 13
# remove suffix '_p'
columns = [c.replace('_p', '') for c in columns]
# remove suffix '_d'
columns = [c.replace('_d', '') for c in columns]
# remove suffix '_u'
# columns = ??


    
# this operation is to remove duplicates
columns = list(set(columns))

In [ ]:
# Code Cell 14
# visualize the first 10 areas
columns[:10]

Para comprender mejor los datos en la base de datos es preciso visualizar una parte de ellos. En particular, queremos seleccionar las tres estadísticas de un área. En primer lugar, escojamos la primer área que tenemos en el conjunto de datos:

In [ ]:
# Code Cell 15
area = columns[0]
print(area) 

Ahora queremos mostrar los datos en relación con el área ‘E07000101’. Necesitamos seleccionar las columnas que contienen las tres mediciones para esta área. Para hacerlo, es necesario manipular las cadenas nuevamente. 

Una forma de formatear una cadena es:

    ‘Estamos considerando {} zona y {} estadísticas’.format (1, 3)
    
    Estamos considerando 1 zona y 3 estadísticas

Debemos agregar los sufijos de nuevo para obtener los nombres de columna correctos. Lo primero que debemos hacer es crear un diccionario que asocie el nombre completo de la medición al sufijo. También será útil escribir una leyenda significativa en el diagrama.

In [ ]:
# Code Cell 16
suffix = {'_p':'ping', '_d':'download', '_u':'upload'}
# we have the suffixes as the keys of the dictionary
print(suffix.keys())

# we have the complete names as the values of the dictionary
print(suffix.values())

Ahora el proceso es dar vuelta las claves del diccionario, escribir y ejecutar la consulta para seleccionar la medición en relación con nuestra área y trazarla usando este código:

    plt.plot(cur.fetchall(), label=suffix[s])
    
El método ‘fetchall()’ recupera todas las filas del resultado de consulta. El campo ‘label’ se utilizará para producir la leyenda del diagrama. También podemos agregar un título al diagrama, mediante el método ‘title()’.

###### b) Escriba la consulta para seleccionar las columnas relevantes.

In [ ]:
# Code Cell 17
area = columns[0]
plt.figure(figsize=(10, 8))

# Plot each variable in suffix.keys() for each area
for s in suffix.keys():
    # query = ??
    # Execute query
    # ...
    plt.plot(cur.fetchall(), label=suffix[s])
plt.legend()
plt.title(area)

## Parte 3: Manipular los datos con pandas

El objetivo es ahora calcular la velocidad promedio de WiFi para todas las áreas del conjunto de datos. Los pandas pueden ayudar en el almacenamiento y la visualización de los datos fácilmente. 

#### Paso 1: Convierta todas las mediciones a promedios.
Para cada autoridad local, queremos saber cuál era la velocidad promedio de Internet para el ping, la carga y la descarga en el período de tiempo en el que nuestros datos fueron recopilados. Los dataframes de pandas son útiles para este tipo de operaciones.

###### a) Cree un dataframe vacío con columnas específicas.

     ['Area', 'Average_p', 'Average_u', 'Average_d']


In [ ]:
# Code Cell 18
#new_columns = ?
#df = pd.DataFrame(columns = ?)


Una medición interesante es la velocidad promedio de ping, carga y descarga para cada área. Para computarla, necesitamos dar vuelta las columnas, y para cada área, para cada estadística, queremos calcular el promedio. Luego lo insertaremos en el dataframe vacío que acabamos de crear. 

Para hacerlo, crearemos una lista temporal y la completaremos con las tres mediciones para cada área. Esta será la fila que insertaremos en el dataframe. 

Para actualizar los comandos:

* Dé vuelta la lista que contiene el nombre de las áreas.
* Dé vuelta los tres sufijos (claves del diccionario ‘suffix’).
* Escriba la consulta para seleccionar el promedio de la medición para un área, mediante los formateadores de cadena.
* Ejecute la consulta y conecte el resultado (método ‘fetchone()’).
* Añada el promedio a la lista temporal (método ‘append()’).
* En el extremo del bucle interno, inserte la lista temporal en el dataframe (método ‘append()’, parámetro ‘ignore_index=True’).

###### b) Calcule los promedios.
Escriba la consulta SQL que computa el promedio de una columna y almacénela en una lista temporal. La lista será la fila nueva adjunta al dataframe. Esto tomará aproximadamente 1 minuto en el RaPi.

In [ ]:
# Code Cell 19
# this will take a while...
for i in range(len(columns)-1): #EDL : replace xrange with range
    tmp_list = []
    tmp_list.append(columns[i])
    for s in suffix.keys():
        # query = ??

        
        cur.execute(query)
        
        mean = cur.fetchone()
        tmp_list.append(mean[0])
    #append the columns to the empty DataFrame
    df = df.append(pd.Series(tmp_list, index=new_columns), ignore_index=True)
# visualize the head of the dataframe here


Es posible mostrar los promedios para cada área con ‘plot()’. Esta sección mostrará si la velocidad promedio es la misma en las diferentes áreas o no.

In [ ]:
# Code Cell 20
plt.figure(figsize=(20,10))
plt.plot(df.index, df[['Average_d','Average_u','Average_p']], 'o')

Ahora las velocidades promedio se almacenan en un dataframe de pandas. Podría resultar útil guardarlos en una tabla en nuestra base de datos, y los pandas ofrecen un método (‘to_sql()’) para hacerlo. 
En primer lugar, verifique si ya existe la tabla (esto puede crear problemas). Si existe, queremos descartarla y luego crearla. Si no, la crearemos directamente. 

El constructo ‘try... except’ capturará la excepción elevada en caso de que la tabla no exista y procederá a su creación (para obtener ayuda, consulte la Sección 8.3 de https://docs.python.org/2.7/tutorial/errors.html). 

###### c) Guarde el trabajo.
Guarde el nuevo dataframe en una tabla de la base de datos InternetSpeed. 

In [ ]:
# Code Cell 21
# Make sure to drop the table first if it already exists
try:
    cur.execute('DROP TABLE average_speed')
except:
    pass

# what is the name of the table you want to save? To what connection?
# Save the dataframe table into a SQL table
# df.to_sql(?,?)


In [ ]:
# Code Cell 22
# just checking everything worked fine: select everything from the table we just created
# and then print the first two rows of the result

query_2 = 'SELECT * FROM average_speed'
cur.execute(query_2)

# print the first fetched record
print(cur.fetchone())
# print the next fetched record
print(cur.fetchone())

## Parte 4: Unir las tablas

Para comprender por qué y cómo cambia la velocidad promedio de Internet alrededor de las áreas, es necesario obtener más información sobre ellas. En particular, estamos interesados en los datos demográficos, para ver si un área con mucha población tiene un servicio de Internet mejor o peor. 

#### Paso 1: Obtenga información relevante.
Esta información se puede encontrar en este archivo: 

    ./Data/LA_population.csv
    
Aquí, LA significa autoridad local, en lugar de Los Ángeles. 
Queremos unir la información contenida en este archivo con la velocidad promedio computada anteriormente.
Primero, haga una nueva tabla y guarde estos datos en la base de datos InternetSpeed.

###### a) Cree la tabla de la población.
Con csvsql, importe el archivo LA_population.csv a la base de datos InternetSpeed. Repita los pasos realizados para crear las otras tablas en la base de datos.

In [ ]:
# Code Cell 23
#first close the database file, otherwise the external csvsql command cannot use
conn.close()

In [ ]:
# Code Cell 24
# use the external csvsql command to add to the DB file data from the CSV file
#!csvsql --db sqlite:///...



In [ ]:
# Code Cell 25
#reopen the DB file
#conn = sqlite3.connect(...)


cur = conn.cursor()

###### b) Compruebe que todo haya funcionado.
Para comprobar que todo haya funcionado y tener una idea de qué hay dentro de esta nueva tabla, seleccione las primeras 10 filas e imprímalas.

In [ ]:
# Code Cell 26
#query = ?


#cur.execute(?)



for row in cur:
    # print the first fetched record


Podemos unir las dos tablas para tener toda la información necesaria. La idea es que las dos tablas tengan un campo en común, el nombre del área. Esta es la clave sobre la que queremos unir. Esto significa que queremos una nueva tabla que tenga una fila para cada nombre de área que contenga información de velocidad promedio e información geográfica. La forma en que SQL lo hace es comparar fila por fila, y fusionar las filas con el mismo valor para la clave (es decir, el mismo nombre de área).

###### c) Ejecute la consulta UNIR e imprima las primeras 10 filas del resultado. 

In [ ]:
# Code Cell 27
query = 'SELECT * FROM average_speed JOIN LA_population ON LA_population."LA_code"=average_speed.Area'

cur.execute(query)
k = 0
for row in cur:
    if k>10:
        break
    #print ?
    

    
    k+=1

<font size='0.5'>&copy; 2017 Cisco y/o sus filiales. Todos los derechos reservados. Este documento es información pública de Cisco.<font>